# User Behavior Analytics Project for Expedia

In [1]:
import pandas as pd
import numpy as np
from statsmodels.formula.api import ols
from scipy.optimize import minimize
from scipy.optimize import LinearConstraint
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report as report
from statsmodels.discrete.discrete_model import Logit
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as qda
from sklearn.ensemble import RandomForestClassifier as rfc

/Users/liuyikun/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
expe = pd.read_csv('B8816-TakeHomeFinal-question4.csv')

### Data cleaning

Main two steps:

1. Drop columns that are not very informative towards the clicking and booking prediction

Particularly, for competitors’ data, after calculation, there is no competitor that has both lower price and availability, so we can remove the 24 columns with competitors’ data.

2. Drop null data reasonably

In [3]:
# drop columns that probably won't contribute much information to the clicking and booking prediction
drop_columns = ['date_time','site_id','visitor_location_country_id','visitor_hist_starrating','visitor_hist_adr_usd','prop_id','prop_country_id','prop_log_historical_price','srch_destination_id','srch_query_affinity_score','orig_destination_distance','position','random_bool']
expe = expe.drop(columns = drop_columns)

In [4]:
expe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 41 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   srch_id                   200000 non-null  int64  
 1   prop_starrating           200000 non-null  int64  
 2   prop_review_score         199719 non-null  float64
 3   prop_brand_bool           200000 non-null  int64  
 4   prop_location_score1      200000 non-null  float64
 5   prop_location_score2      156405 non-null  float64
 6   price_usd                 200000 non-null  float64
 7   promotion_flag            200000 non-null  int64  
 8   srch_length_of_stay       200000 non-null  int64  
 9   srch_booking_window       200000 non-null  int64  
 10  srch_adults_count         200000 non-null  int64  
 11  srch_children_count       200000 non-null  int64  
 12  srch_room_count           200000 non-null  int64  
 13  srch_saturday_night_bool  200000 non-null  i

In [5]:
# Since there are many null data in competitors' data, we have to integrate some infomation.
# calculate number of competitors with lower price and avalibility

columns1 = ['comp1_rate','comp2_rate','comp3_rate','comp4_rate','comp5_rate','comp6_rate','comp7_rate','comp8_rate']
columns2 = ['comp1_inv','comp2_inv','comp3_inv','comp4_inv','comp5_inv','comp6_inv','comp7_inv','comp8_inv']
expe['Num_comp'] = ((expe[columns1] == 1) &(expe[columns2] == -1)).sum(axis = 1)

In [6]:
expe['Num_comp'].value_counts()

0    200000
Name: Num_comp, dtype: int64

In [7]:
# It turns out that no competitor have both avalibility and lower price, which means we don't have to worry about competitors.
columns3 = ['comp1_rate_percent_diff','comp2_rate_percent_diff','comp3_rate_percent_diff','comp4_rate_percent_diff','comp5_rate_percent_diff','comp6_rate_percent_diff','comp7_rate_percent_diff','comp8_rate_percent_diff']
expe = expe.drop(columns = columns1)
expe = expe.drop(columns = columns2)
expe = expe.drop(columns = columns3)
expe = expe.drop(columns = ['Num_comp'])

In [8]:
# deal with null data
# The 'gross_bookings_usd' indicates similar information of 'price_usd' and 'booking_bool'. Drop this column.
# The 'prop_location_score1' is probably enough to indicate the location desirablity, so drop 'prop_location_score2' as it contains a lot of null data.
# Drop rows that still contain null data.
expe = expe.drop(columns = ['gross_bookings_usd','prop_location_score2'])
expe = expe.dropna()

In [9]:
expe

,srch_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,price_usd,promotion_flag,srch_length_of_stay,srch_booking_window,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,click_bool,booking_bool
0,1,3,3.5,1,2.83,104.77,0,1,0,4,0,1,1,0,0
1,1,4,4.0,1,2.20,170.74,0,1,0,4,0,1,1,0,0
2,1,3,4.5,1,2.20,179.80,0,1,0,4,0,1,1,0,0
3,1,2,4.0,1,2.83,602.77,0,1,0,4,0,1,1,0,0
4,1,4,3.5,1,2.64,143.58,0,1,0,4,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,13431,4,4.5,1,1.10,143.00,0,2,11,1,1,1,1,0,0
199996,13431,3,3.5,1,1.79,105.00,0,2,11,1,1,1,1,0,0
199997,13431,3,4.0,1,1.10,146.00,0,2,11,1,1,1,1,0,0
199998,13431,4,4.0,1,4.08,259.00,0,2,11,1,1,1,1,0,0


### Data exploration

In [10]:
# 20% of the properties have promotion displayed.
expe.promotion_flag.value_counts()

0    156828
1     42891
Name: promotion_flag, dtype: int64

In [11]:
# Nearly half of the records cover weekend reservations.
expe.srch_saturday_night_bool.value_counts()

0    100163
1     99556
Name: srch_saturday_night_bool, dtype: int64

In [12]:
# The mean of property review score is 3.8.
np.mean(expe.prop_review_score)

3.7988098278080704

In [13]:
# The mean of property star rating is 3.2.
np.mean(expe.prop_starrating)

3.198799313034814

In [14]:
expe.click_bool.value_counts()

0    190777
1      8942
Name: click_bool, dtype: int64

Only 4% of the properties are clicked.

This ratio is very low, which means that if we fit a model with this unbalanced dataset, it is very likely that the click_bool prediction from the model will all be 0. Also as clicking and booking is more valuable than not for Expedia, the cost function should be altered so that the false negative will be penalized more. So we have to do resampling. 

In [15]:
# Among all the properties that are clicked, about 63% of the properties are booked, which is quite a high ratio.
expe[expe.click_bool == 1].booking_bool.value_counts()

1    5593
0    3349
Name: booking_bool, dtype: int64

In [16]:
# If the property is under promotion, the click ratio is higher than those not.
expe.groupby('promotion_flag').mean().click_bool

promotion_flag
0    0.040682
1    0.059733
Name: click_bool, dtype: float64

In [17]:
# When review score in range (3.5,4,5), the click-on ratio is higher than average.
expe.groupby('prop_review_score').mean().click_bool

prop_review_score
0.0    0.029910
1.0    0.014493
1.5    0.021700
2.0    0.024558
2.5    0.034458
3.0    0.034748
3.5    0.045280
4.0    0.050463
4.5    0.047008
5.0    0.033617
Name: click_bool, dtype: float64

In [18]:
# For each people seearching on Expedia, how many properties do they look through?
expe.groupby('srch_id').count().click_bool.value_counts()

32    1204
33    1045
31     825
30     476
34     383
27     262
28     262
29     253
6      212
5      190
8      171
26     170
15     167
9      161
11     157
25     148
10     148
13     147
20     147
21     144
12     141
24     139
16     135
18     134
7      134
19     131
17     130
23     128
22     124
14     123
35      72
36       4
37       1
4        1
Name: click_bool, dtype: int64

In [19]:
# For each people seearching on Expedia, how many properties do they click?
expe.groupby('srch_id').sum().click_bool.value_counts()

1     7547
2      352
3       86
4       34
5       12
0       11
6        9
7        5
10       5
11       2
19       1
15       1
13       1
9        1
12       1
8        1
Name: click_bool, dtype: int64

## Build a model that predicts the likelihood a property will be clicked upon

### Data preparation

In [20]:
# Transform 'prop_review_score' into a boolean variable: 'popular_score'. 
# popular_score = 1 if 'prop_review_score' in range(3.5,4,5); popular_score = 0 otherwise.
expe['popular_score'] = np.where((expe.prop_review_score<=4.5) & (expe.prop_review_score >=3.5),1,0)

In [21]:
# Split the dataset into training and validation sets
train_c,valid_c = train_test_split(expe,test_size = 0.1,random_state = 1)

In [22]:
# Resampling training and validation set: under-sample click_bool = 0 by 1/10 and over-sample click_bool = 1 by 3 times.
train_c1 = train_c[train_c.click_bool == 1]
train_c0 = train_c[train_c.click_bool == 0]
n1 = len(train_c1)
n0 = len(train_c0)
train_c1 = train_c1.sample(3*n1,replace=True)
train_c0 = train_c0.sample(int(n0/10), replace=True)
train_c = pd.concat([train_c0, train_c1], axis=0)

valid_c1 = valid_c[valid_c.click_bool == 1]
valid_c0 = valid_c[valid_c.click_bool == 0]
m1 = len(valid_c1)
m0 = len(valid_c0)
valid_c1 = valid_c1.sample(3*m1,replace=True)
valid_c0 = valid_c0.sample(int(m0/10), replace=True)
valid_c = pd.concat([valid_c0, valid_c1], axis=0)

In [23]:
train_xc = train_c.drop(columns = ['srch_id','click_bool','booking_bool'])
train_yc = train_c['click_bool']
valid_xc = valid_c.drop(columns = ['srch_id','click_bool','booking_bool'])
valid_yc = valid_c['click_bool']

### Fit a logistic regression model

In [24]:
model_logit_c = Logit(train_yc,train_xc).fit()

Optimization terminated successfully.
         Current function value: 0.670478
         Iterations 4


In [25]:
model_logit_c.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:             click_bool   No. Observations:                41432
Model:                          Logit   Df Residuals:                    41419
Method:                           MLE   Df Model:                           12
Date:                Mon, 09 Mar 2020   Pseudo R-squ.:                 0.01165
Time:                        20:41:49   Log-Likelihood:                -27779.
converged:                       True   LL-Null:                       -28107.
Covariance Type:            nonrobust   LLR p-value:                1.790e-132
============================================================================================
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
prop_starrating              0.0591      0.010      5.637      0.000       0.039       0.080
prop_review_score           -0.0290      0.012     -2.496      0.013      -0.052      -0.006
prop_brand_bool             -0.0343      0.022     -1.567      0.117      -0.077       0.009
prop_location_score1        -0.0459      0.007     -6.581      0.000      -0.060      -0.032
price_usd                -8.855e-06   5.45e-06     -1.624      0.104   -1.95e-05    1.83e-06
promotion_flag               0.3956      0.024     16.234      0.000       0.348       0.443
srch_length_of_stay         -0.0070      0.005     -1.358      0.175      -0.017       0.003
srch_booking_window       -8.82e-05      0.000     -0.451      0.652      -0.000       0.000
srch_adults_count           -0.0096      0.013     -0.745      0.456      -0.035       0.016
srch_children_count          0.0412      0.014      2.965      0.003       0.014       0.068
srch_room_count              0.0177      0.026      0.676      0.499      -0.034       0.069
srch_saturday_night_bool     0.0291      0.021      1.412      0.158      -0.011       0.070
popular_score                0.3937      0.032     12.470      0.000       0.332       0.456
============================================================================================
"""

In [26]:
train_xc1 = train_xc[['prop_starrating','prop_review_score','promotion_flag','srch_room_count','popular_score']]
valid_xc1 = valid_xc[['prop_starrating','prop_review_score','promotion_flag','srch_room_count','popular_score']]

In [27]:
model_logit_c1 = Logit(train_yc,train_xc1).fit()

Optimization terminated successfully.
         Current function value: 0.671260
         Iterations 4


In [28]:
pred_yc1 = model_logit_c1.predict(valid_xc1)
pred_yc1 = np.where(pred_yc1<=0.5,0,1)
print(report(valid_yc,pred_yc1))

              precision    recall  f1-score   support

           0       0.54      0.15      0.23      1911
           1       0.59      0.91      0.71      2559

    accuracy                           0.58      4470
   macro avg       0.57      0.53      0.47      4470
weighted avg       0.57      0.58      0.51      4470



### Fit a quadratic discriminant model

In [29]:
model_qda_c = qda().fit(train_xc,train_yc)

In [30]:
pred_yc2 = model_qda_c.predict(valid_xc)
print(report(valid_yc,pred_yc2))

              precision    recall  f1-score   support

           0       0.52      0.20      0.28      1911
           1       0.59      0.86      0.70      2559

    accuracy                           0.58      4470
   macro avg       0.55      0.53      0.49      4470
weighted avg       0.56      0.58      0.52      4470



### Fit a random forest tree

In [31]:
model_rf_c = rfc(max_features = 13).fit(train_xc,train_yc)

In [32]:
pred_yc3 = model_rf_c.predict(valid_xc)
print(report(valid_yc,pred_yc3))

              precision    recall  f1-score   support

           0       0.44      0.77      0.56      1911
           1       0.61      0.27      0.38      2559

    accuracy                           0.48      4470
   macro avg       0.53      0.52      0.47      4470
weighted avg       0.54      0.48      0.45      4470



Because the “clicking” is more important than “not clicking”, which means false negative should be penalized more than false negative, we should use a metrics other than misclassification error to select a good model. I chose recall, true positive / (true positive + false negative) as the most important metrics, and specificity, true negative / (true negative + false positive) as the second important metrics. These 2 metrics have a trade-off.

The model selected is the __quadratic discriminant model__, with a recall of 0.86 and a specificity of 0.20.

## Build a model that predicts the likelihood a property that has been clicked upon will be booked

In [33]:
expe_click = expe[expe.click_bool == 1]

In [34]:
# Split the dataset into training and validation sets
train_b,valid_b = train_test_split(expe_click,test_size = 0.1,random_state = 1)

In [35]:
train_xb = train_b.drop(columns = ['srch_id','click_bool','booking_bool'])
train_yb = train_b['booking_bool']
valid_xb = valid_b.drop(columns = ['srch_id','click_bool','booking_bool'])
valid_yb = valid_b['booking_bool']

### Fit a logit model

In [36]:
model_logit_b = Logit(train_yb,train_xb).fit()
model_logit_b.summary()

Optimization terminated successfully.
         Current function value: 0.625988
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:           booking_bool   No. Observations:                 8047
Model:                          Logit   Df Residuals:                     8034
Method:                           MLE   Df Model:                           12
Date:                Mon, 09 Mar 2020   Pseudo R-squ.:                 0.05234
Time:                        20:47:47   Log-Likelihood:                -5037.3
converged:                       True   LL-Null:                       -5315.6
Covariance Type:            nonrobust   LLR p-value:                2.076e-111
============================================================================================
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
prop_starrating             -0.0060      0.026     -0.235      0.814      -0.056       0.044
prop_review_score            0.1679      0.029      5.817      0.000       0.111       0.224
prop_brand_bool              0.1684      0.051      3.281      0.001       0.068       0.269
prop_location_score1        -0.0266      0.016     -1.618      0.106      -0.059       0.006
price_usd                 6.315e-06   1.75e-05      0.360      0.719    -2.8e-05    4.07e-05
promotion_flag               0.4211      0.056      7.521      0.000       0.311       0.531
srch_length_of_stay         -0.1877      0.014    -13.220      0.000      -0.216      -0.160
srch_booking_window         -0.0039      0.000     -8.531      0.000      -0.005      -0.003
srch_adults_count           -0.1473      0.032     -4.672      0.000      -0.209      -0.086
srch_children_count         -0.0552      0.032     -1.705      0.088      -0.119       0.008
srch_room_count              0.3893      0.070      5.559      0.000       0.252       0.527
srch_saturday_night_bool    -0.1179      0.049     -2.389      0.017      -0.215      -0.021
popular_score                0.3203      0.078      4.097      0.000       0.167       0.473
============================================================================================
"""

In [37]:
train_xb1 = train_xb.drop(columns = ['prop_starrating','prop_location_score1','price_usd','srch_children_count'])
valid_xb1 = valid_xb.drop(columns = ['prop_starrating','prop_location_score1','price_usd','srch_children_count'])

model_logit_b2 = Logit(train_yb,train_xb1).fit()

Optimization terminated successfully.
         Current function value: 0.626365
         Iterations 5


In [38]:
pred_yb1 = model_logit_b2.predict(valid_xb1)
pred_yb1 = np.where(pred_yb1<=0.5,0,1)
print(report(valid_yb,pred_yb1))

              precision    recall  f1-score   support

           0       0.61      0.24      0.35       347
           1       0.65      0.90      0.76       548

    accuracy                           0.65       895
   macro avg       0.63      0.57      0.55       895
weighted avg       0.64      0.65      0.60       895



### Fit a quadratic discriminant model

In [39]:
model_qda_b = qda().fit(train_xb,train_yb)

In [40]:
pred_yb2 = model_qda_b.predict(valid_xb)
print(report(valid_yb,pred_yb2))

              precision    recall  f1-score   support

           0       0.39      0.98      0.56       347
           1       0.77      0.04      0.07       548

    accuracy                           0.40       895
   macro avg       0.58      0.51      0.32       895
weighted avg       0.62      0.40      0.26       895



### Fit a random forest tree

In [41]:
model_rf_b = rfc(max_features = 13).fit(train_xb,train_yb)

In [42]:
pred_yb3 = model_rf_b.predict(valid_xb)
print(report(valid_yb,pred_yb3))

              precision    recall  f1-score   support

           0       0.54      0.41      0.47       347
           1       0.68      0.78      0.72       548

    accuracy                           0.64       895
   macro avg       0.61      0.60      0.60       895
weighted avg       0.62      0.64      0.62       895



The model selected is the __random forest tree__, with a recall of 0.78 and a specificity of 0.41.

### A scoring model that allows Expedia to rank order the list of properties for each customer based on the expected gross booking revenue:

- Use clustering models to define different customer segments.
- Train probability estimation models for different segments of customers just like (b) and (c) from Expedia webpage data. As we can see from this problem, the features of these models are combinations of customer features (such as countries, number of nights, etc.), and property features (such as star rating, review ranking, etc.).
- When having a certain customer, first use his/her history to determine which customer segment he/she belongs to. Then, run the suitable models. If this is a new customer, use a general model trained from all customer data.
- This chosen model can estimate the probability whether a customer will book a certain property. Therefore, the expected revenue of this certain property from this customer can be estimated as price * probability. The scoring model takes this estimated revenue as the key factor to rank different properties for this customer.